## ANA - Nuclear NICD Measurement

### Notes

- The goal of this analysis is to fit a model characterizing import and export of opto-Notch in light vs. darkness
    - The data for model fitting is from His::GFP experiments performed as follows
        - Full imaging for 10min ("light on" condition)
        - Either 5min or 10min of darkness ("light off" condition; no live data!)
        - A single endpoint after the period of darkness
    - The fitted model is packaged as a function for use in the motif modeling
        - This is done so that the models have realistic input functions based on when the activating light is on/off
        
        
- This notebook contains the following steps
    1. Data preparation: loading, background subtraction, ratio calculation
    2. Model derivation: analytical solution of 2-compartment model with bleaching using `sympy`
        - Note that the bleaching constant is derived from a separate experiment
    3. Model fitting: based on minimizing least squares of nuclear/total intensity ratios
        - The ratios are used to normalize the data for absolute intensity differences and cell movement (in z)

### Prep

In [ ]:
# Imports

import os
import inspect
import pickle, dill

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sympy as sym
from sympy import symbols, dsolve, Function, Derivative, Eq
from sympy.utilities.lambdify import lambdify

from scipy.optimize import minimize

### Data Preparation

#### Data Loading

In [ ]:
# Find all relevant dirs and files

# Get all file paths
walk = os.walk('../Data/Measurements/HisGFP')

# Create a dict for all samples
path_dict = dict()
for w in walk:
    for f in w[-1]:
        if f.endswith('_NICDquant_ON.tsv'):
            path_dict[w[0]] = dict()

# Parse out relevant files
for dpath in path_dict:
    for f in os.listdir(dpath):
        if f.endswith('_NICDquant_ON.tsv'):
            path_dict[dpath]['on_fname'] = f
        elif f.endswith('_NICDquant_OFF.tsv'):
            path_dict[dpath]['off_fname'] = f
    if len(path_dict[dpath]) != 2:
        raise Exception("Incorrect number of files found for this path!")

# Report
for dpath in path_dict:
    print(dpath)
    for fkey in path_dict[dpath]:
        print('    {:<8}'.format(fkey), '--', path_dict[dpath][fkey])

In [ ]:
# Load the data

data_dict = {}
for dpath in path_dict:
    
    # Get information on the endpoint
    if '10min export' in dpath:
        export_time = 'exp10min'
    elif '5min export' in dpath:
        export_time = 'exp5min'
    else:
        raise Exception('No proper export time?')
    
    # Load the data
    sample_num = os.path.split(dpath)[-1]
    
    with open(os.path.join(dpath, path_dict[dpath]['on_fname']), 'r') as infile:
        header = infile.readline().strip().split('\t')
    
    on_data  = pd.read_csv(os.path.join(dpath, path_dict[dpath]['on_fname']), sep='\t')
    off_data = pd.read_csv(os.path.join(dpath, path_dict[dpath]['off_fname']), sep='\t')
    
    data_dict[export_time+'_'+sample_num] = {'on'  : on_data, 'off' : off_data}
    
# Report 
print('Header:', ', '.join(header), '\n')
for dkey in data_dict:
    print(dkey)
    for dset in data_dict[dkey]:
        print('  --', dset + ':', data_dict[dkey][dset].shape)

In [ ]:
# Plot the raw data

plt.figure(figsize=(10,6))

for di, dkey in enumerate(data_dict):
    
    # Plot `on`
    plt.plot(data_dict[dkey]['on']['time_step'], 
             data_dict[dkey]['on']['nuc_on_nicd_mean'],
             c='teal', alpha=0.4, 
             label='nuc on' if di==0 else '_no-label_')
    plt.plot(data_dict[dkey]['on']['time_step'], 
             data_dict[dkey]['on']['cyt_on_nicd_mean'],
             c='darkorange', alpha=0.4, 
             label='cyt on' if di==0 else '_no-label_')
    
    # Plot `off`
    export_time = {'exp5min':15, 'exp10min':20}[dkey.split('_')[0]]
    plt.plot([10.0, export_time], 
             [data_dict[dkey]['on']['nuc_on_nicd_mean'].values[-1],
              data_dict[dkey]['off']['nuc_off_nicd_mean']],
             '--', c='teal', alpha=0.4, 
             label='nuc off' if di==0 else '_no-label_')
    plt.plot([10.0, export_time], 
             [data_dict[dkey]['on']['cyt_on_nicd_mean'].values[-1],
              data_dict[dkey]['off']['cyt_off_nicd_mean']],
             '--', c='darkorange', alpha=0.4, 
             label='cyt off' if di==0 else '_no-label_')

# Cosmetics
plt.legend(loc='upper right', frameon=False, ncol=2)
plt.xlabel('time [min]')
plt.ylabel('Mean NICD signal [au]')
    
plt.show()

#### Rationale for area-corrected ratiometric normalization

- The problem
    - The goal is to normalize out variation due to absolute intensity and cell movements (in z; affects mask areas)
    - So we want the "nuclear over total signal ratio", i.e. `nuclear / (nuclear + cytoplasmic)`
    - But this wouldn't work: `mean(nuclear) / (mean(nuclear) + mean(cytoplasmic))`
        - Because the addition in this case doesn't reflect the total signal (as the relative area is not considered)
    - Yet this also wouldn't work: `sum(nuclear) / (sum(nuclear) + sum(cytoplasmic)`
        - Because varying areas over time would affect the ratio, which they should not


- The solution is to use the "ratio of concentrations"
    - Compartmental concentration is measured as `[Nuc] = sum(nuc) / area(nuc) = mean(nuc)`
    - Total concentration is `[Tot] = (sum(nuc) + sum(cyt)) / (area(nuc) + area(cyt))`
        - Note that in the implementation the sums are reconstructed from the means following background subtraction
    - Relative concentration is the ratio we want, i.e. `cR = [Nuc] / [Tot]`
    - More completely: $cR = \frac{[NICD_{nuc}]}{[NICD_{tot}]} = \frac{mean(S_{nuc})}{mean(S_{nuc}+S_{cyt})} = \frac{\frac{S_{nuc}}{A_{nuc}}}{\frac{S_{nuc} + S_{cyt}}{A_{nuc} + A_{cyt}}} = \frac{S_{nuc}}{S_{nuc}+S_{cyt}} \cdot \frac{A_{nuc}+A_{cyt}}{{A_{nuc}}} = \frac{S_{nuc}}{S_{all}} \cdot \frac{A_{all}}{A_{nuc}}$


- The resulting numbers are not completely intuitive, as they are a "relative concentration"
    - However, there is some very sensible behavior, such as balancing:
        - `Sn=1, Sc=1, An=1, Ac=1 -> cR = 2/1*1/2 = 1.0`
        - `Sn=2/3, Sc=1/3, An=2/3, Ac=1/3 -> cR = 1.0`
    - Other examples are a less intuitive, but they mathematically hold:
        - `Sn=2/3, Sc=1/3, An=1/2, Ac=1/2 -> cR = 4/3 = 1.333`
        - `Sn=2/3, Sc=1/3, An=1/3, Ac=2/3 -> cR = 2.0`
    - More importantly, this ratio can be recapitulated from model equations for `[N]` and `[C]`, namely:
        - $cR = \frac{[NICD_{nuc}]}{[NICD_{tot}]} = \frac{[N]}{[N+C]} = \frac{(N/1)}{(N+C)/(1+1)} = \frac{N}{(N+C)/2}$
    - Finally, this empirically results in very similar tracks across multiple samples, normalizing out apparent variability (see plot below)


- Thus, `cR` is being used for all further analysis / model fitting

#### Background correction & ratio calculation

In [ ]:
for dkey in data_dict:
    
    # Use lowest mean value as simple background
    mean_bg = np.min(np.concatenate([data_dict[dkey]['on']['nuc_on_nicd_mean'], 
                                     data_dict[dkey]['off']['nuc_off_nicd_mean']]))
    
    
    # For `on` time courses
    
    # Subtract background
    data_dict[dkey]['on']['nuc_on_nicd_mean'] = data_dict[dkey]['on']['nuc_on_nicd_mean'] - mean_bg
    data_dict[dkey]['on']['cyt_on_nicd_mean'] = data_dict[dkey]['on']['cyt_on_nicd_mean'] - mean_bg
    
    # Compute area-corrected ratio (cR)
    nuc_ni = data_dict[dkey]['on']['nuc_on_nicd_mean']
    cyt_ni = data_dict[dkey]['on']['cyt_on_nicd_mean']
    nuc_sz = data_dict[dkey]['on']['nuc_on_area']
    cyt_sz = data_dict[dkey]['on']['cyt_on_area']
    
    data_dict[dkey]['on']['all_on_nicd_mean'] = (nuc_ni*nuc_sz + cyt_ni*cyt_sz) / (nuc_sz + cyt_sz)
    
    data_dict[dkey]['on']['cR_on_nuc-all'] = nuc_ni / data_dict[dkey]['on']['all_on_nicd_mean']
    data_dict[dkey]['on']['cR_on_cyt-all'] = cyt_ni / data_dict[dkey]['on']['all_on_nicd_mean']
    
    
    # For `off` endpoints
    
    # Subtract background
    data_dict[dkey]['off']['nuc_off_nicd_mean'] = data_dict[dkey]['off']['nuc_off_nicd_mean'] - mean_bg
    data_dict[dkey]['off']['cyt_off_nicd_mean'] = data_dict[dkey]['off']['cyt_off_nicd_mean'] - mean_bg
    
    # Compute area-corrected ratio (cR)
    nuc_ni = data_dict[dkey]['off']['nuc_off_nicd_mean']
    cyt_ni = data_dict[dkey]['off']['cyt_off_nicd_mean']
    nuc_sz = data_dict[dkey]['off']['nuc_off_area']
    cyt_sz = data_dict[dkey]['off']['cyt_off_area']
    
    data_dict[dkey]['off']['all_off_nicd_mean'] = (nuc_ni*nuc_sz + cyt_ni*cyt_sz) / (nuc_sz + cyt_sz)
    
    data_dict[dkey]['off']['cR_off_nuc-all'] = nuc_ni / data_dict[dkey]['off']['all_off_nicd_mean']
    data_dict[dkey]['off']['cR_off_cyt-all'] = cyt_ni / data_dict[dkey]['off']['all_off_nicd_mean']

In [ ]:
# Plot the prepared data

for di, dkey in enumerate(data_dict):
    
    # Plot `on`
    plt.plot(data_dict[dkey]['on']['time_step'], 
             data_dict[dkey]['on']['cR_on_nuc-all'],
             c='teal', alpha=0.4, 
             label='nuc' if di==0 else '_no-label_')
    plt.plot(data_dict[dkey]['on']['time_step'], 
             data_dict[dkey]['on']['cR_on_cyt-all'],
             c='darkorange', alpha=0.4, 
             label='cyt' if di==0 else '_no-label_')
    
    # Plot `off` endpoints
    export_time = {'exp5min':15, 'exp10min':20}[dkey.split('_')[0]]
    plt.scatter(export_time, 
                data_dict[dkey]['off']['cR_off_nuc-all'],
                c='teal', alpha=0.4)
    plt.scatter(export_time, 
                data_dict[dkey]['off']['cR_off_cyt-all'],
                c='darkorange', alpha=0.4)
    
    # Add the last tp of `on` as first of `off`
    plt.scatter(data_dict[dkey]['on']['time_step'].iloc[-1], 
                data_dict[dkey]['on']['cR_on_nuc-all'].iloc[-1],
                c='teal', alpha=0.4)
    plt.scatter(data_dict[dkey]['on']['time_step'].iloc[-1], 
                data_dict[dkey]['on']['cR_on_cyt-all'].iloc[-1],
                c='darkorange', alpha=0.4)

# Cosmetics
plt.legend(loc='lower center', frameon=False, ncol=2)
plt.xlabel('time [min]')
plt.ylabel('NICD <label>/total ratio (corr) [au]')
    
plt.show()

### Model Derivation

#### Simple Mass-Action Model with Bleaching

- Two compartments
    - Cytoplasmic $C$
    - Nuclear $N$
- Simple mass-action transfer between them
    - Two parameters:
        - Import constant $k_i$ ("in")
        - Explort constant $k_o$ ("out")
    - To be fitted independently for light and dark conditions
- Uniform bleaching in both compartments
    - Split each variable into a light (`l`) and a dark (`d`) version
    - The transfer kinetics remain exactly the same
    - The transition from light to dark is based on an externally defined parameter $b$
        - where `b=0.1029` for ON (see `ANA - nuclear nicd bleaching.ipynb`)
        - and `b=0.0000` for OFF (based on the observation that there is no bleaching when the 488 laser is off)


- System:

$C_l \underset{k_o}{\overset{k_i}{\rightleftharpoons}} N_l$

$C_d \underset{k_o}{\overset{k_i}{\rightleftharpoons}} N_d$

$C_l \overset{b}{\rightarrow} C_d$

$N_l \overset{b}{\rightarrow} N_d$


- Differential equations:

$\frac{dC_l}{dt} = k_o N_l - k_i C_l - b C_l$

$\frac{dN_l}{dt} = k_i C_l - k_o N_l - b N_l$

$\frac{dC_d}{dt} = k_o N_d - k_i C_d + b C_l$

$\frac{dN_d}{dt} = k_i C_d - k_o N_d + b N_l$


- Additional statements:

$C_d(t=0) = 0$

$N_d(t=0) = 0$

#### Symbolic integration with Sympy

In [ ]:
# Define the symbols and functions

ko, ki, b, t = symbols('ko, ki, b, t', positive=True)
C1, C2, C3, C4 = symbols('C1, C2, C3, C4')
Cl, Cd = Function('Cl'), Function('Cd')
Nl, Nd = Function('Nl'), Function('Nd')

# Note: Forcing k and t to be positive simplifies the solution a lot by restricting its domains!

In [ ]:
# Specify the differential equations

eq_Cl = Eq( Cl(t).diff(t), ko*Nl(t) - ki*Cl(t) - b*Cl(t) )
eq_Nl = Eq( Nl(t).diff(t), ki*Cl(t) - ko*Nl(t) - b*Nl(t) )
eq_Cd = Eq( Cd(t).diff(t), ko*Nd(t) - ki*Cd(t) + b*Cl(t) )
eq_Nd = Eq( Nd(t).diff(t), ki*Cd(t) - ko*Nd(t) + b*Nl(t) )

In [ ]:
# Solve the differential equations

Cl_sol, Nl_sol, Cd_sol, Nd_sol = dsolve([eq_Cl, eq_Nl, eq_Cd, eq_Nd])

In [ ]:
display(Cl_sol)
display(Nl_sol)
display(Cd_sol)
display(Nd_sol)

#### Handling the arbitrary constants

Sympy's constants used in the solution (i.e. $C_1$ through $C_4$) are arbitrary.

However, it should be possible to express them in terms of the initial conditions ($C_l(t=0)$, etc.):

In [ ]:
# Get out the constants

constants = sym.solve([Cl_sol.subs({t:0}), Nl_sol.subs({t:0}),
                       Cd_sol.subs({t:0}), Nd_sol.subs({t:0})], 
                      [C1, C2, C3, C4])

In [ ]:
# Substitute the constants with their definition (& simplify)

Cl_rdy = Cl_sol.subs(constants).simplify()
Nl_rdy = Nl_sol.subs(constants).simplify()
Cd_rdy = Cd_sol.subs(constants).simplify()
Nd_rdy = Nd_sol.subs(constants).simplify()

In [ ]:
display(Cl_rdy)
display(Nl_rdy)
display(Cd_rdy)
display(Nd_rdy)

#### Test whether the solutions make sense using example values

In [ ]:
# Define initial values and parameters

subs = {Cl(0):8, Nl(0):2, Cd(0):0, Nd(0):0, ki:4, ko:2, b:0}
subs = {Cl(0):8, Nl(0):2, Cd(0):0, Nd(0):0, ki:4, ko:2, b:0.5}

In [ ]:
# Evaluate the functions over a range of time

t_arr = np.linspace(0.0, 1.0, 100)
Cl_arr = np.empty_like(t_arr)
Nl_arr = np.empty_like(t_arr)
Cd_arr = np.empty_like(t_arr)
Nd_arr = np.empty_like(t_arr)
for i,t_val in enumerate(t_arr):
    subs[t] = t_val
    Cl_arr[i] = Cl_rdy.subs(subs).evalf().args[1]
    Nl_arr[i] = Nl_rdy.subs(subs).evalf().args[1]
    Cd_arr[i] = Cd_rdy.subs(subs).evalf().args[1]
    Nd_arr[i] = Nd_rdy.subs(subs).evalf().args[1]

In [ ]:
# Plot the results

plt.plot(t_arr, Cl_arr + Cd_arr, '-', c='darkblue', alpha=1.0, label='Cyt')
plt.plot(t_arr, Nl_arr + Nd_arr, '-', c='darkred',  alpha=1.0, label='Nuc')
plt.plot(t_arr, Cl_arr, '--',  c='blue', alpha=0.7, label='Cyt-L')
plt.plot(t_arr, Nl_arr, '--',  c='red',  alpha=0.7, label='Nuc-L')
plt.plot(t_arr, Cd_arr, ':', c='darkslategray', alpha=0.7, label='Cyt-D')
plt.plot(t_arr, Nd_arr, ':', c='saddlebrown',  alpha=0.7, label='Nuc-D')

plt.legend()
plt.show()

### Model Fitting


- The ratio retrieved from the data is defined as:
    - $cR = \frac{[N]}{[T]} = \frac{\frac{N}{A_N}}{\frac{N+C}{A_N+A_C}} = \frac{N}{N+C} \cdot \frac{A_N+A_C}{A_N} = \frac{N}{T} \cdot \frac{A_T}{A_N}$
    - Note that the compartment sizes are normalized out


- Now `cR` must also be constructed from the model output
    - The model is built on equal compartment sizes ($A_C=A_N=1$)...
    - ...but what matters are only the concentrations, so the compartment size can again be normalized out at the end:
    - $cR_{model} = \frac{N}{N+C} \cdot \frac{A_N+A_C}{A_N} = \frac{N}{N+C} \cdot \frac{1+1}{1} = \frac{[N]}{[N]+[C]} \cdot 2$
    - Note that $C=[C]$ and $N=[N]$ for $A_C=A_N=1$
    

- To fit the import model, there are 4 free parameters:
    - $N(t=0)$ (initial nuclear presence)
    - $C(t=0)$ (initial cytoplasmic presence)
    - $k_i$ (import rate constant)
    - $k_o$ (export rate constant)
    - Note the following pre-defined parameters:
        - Bleaching (ON): `b=0.1029` (see `ANA - nuclear nicd bleaching.ipynb`)
        - Initial bleached FPs: $C_d(t=0)=0$ and $N_d(t=0)=0$ 
            
            
- When fitting the export model, only 2 free parameters remain:
    - $k_i$ (import rate constant)
    - $k_o$ (export rate constant)
    - All other parameters are now pre-defined:
        - Bleaching (OFF): `b=0.0000` (based on control experiment showing no bleaching with 488 off)
        - All four initial concentrations are forwarded from the end of the import solution

In [ ]:
# Get relevant data (time and ratio)

# For `on`
on_times  = []
on_ratios = []
for dkey in data_dict:
    on_times.append( data_dict[dkey]['on']['time_step'].values)
    on_ratios.append(data_dict[dkey]['on']['cR_on_nuc-all'].values)
on_times  = np.concatenate(on_times)
on_ratios = np.concatenate(on_ratios)

# For `off`
off_times  = []
off_ratios = []
for dkey in data_dict:
    off_times.append(0.0)
    off_times.append({'exp5min':5.0, 'exp10min':10.0}[dkey.split('_')[0]])
    off_ratios.append(data_dict[dkey]['on']['cR_on_nuc-all'].values[-1])
    off_ratios.append(data_dict[dkey]['off']['cR_off_nuc-all'].values)
off_times  = np.array(off_times)
off_ratios = np.array(off_ratios)

# Check
plt.scatter(on_times, on_ratios, color='teal', alpha=0.4)
plt.scatter(off_times+10.0, off_ratios, color='darkorange', alpha=0.4)
plt.xlabel('time [min]')
plt.ylabel('N/T ratios')
plt.show()

#### Fitting for the `on` condition (import)

In [ ]:
# Prepare the equations for fitting

# Show  example
display(Nl_rdy.rhs)

# Replace what is known
Nl_fit_on = Nl_rdy.rhs.subs({Cd(0):0, Nd(0):0, b:0.1029})
Cl_fit_on = Cl_rdy.rhs.subs({Cd(0):0, Nd(0):0, b:0.1029})
Nd_fit_on = Nd_rdy.rhs.subs({Cd(0):0, Nd(0):0, b:0.1029})
Cd_fit_on = Cd_rdy.rhs.subs({Cd(0):0, Nd(0):0, b:0.1029})

# Show example result
display(Nl_fit_on)

# Lambdify
Nl_fit_on_np = lambdify([ki, ko, Nl(0), Cl(0), t], Nl_fit_on, modules="numpy")
Cl_fit_on_np = lambdify([ki, ko, Nl(0), Cl(0), t], Cl_fit_on, modules="numpy")
Nd_fit_on_np = lambdify([ki, ko, Nl(0), Cl(0), t], Nd_fit_on, modules="numpy")
Cd_fit_on_np = lambdify([ki, ko, Nl(0), Cl(0), t], Cd_fit_on, modules="numpy")
print('\nNl_fit_on_np signature:', inspect.signature(Nl_fit_on_np))

In [ ]:
# Loss function

def ratiofit_loss_on(params, t, data):
    
    # Run prediction
    pred_Nl = Nl_fit_on_np(*params, t)
    pred_Cl = Cl_fit_on_np(*params, t)
    pred_R = pred_Nl / (pred_Nl + pred_Cl) * 2
    
    # Compute mean square error
    MSE = np.mean((data - pred_R)**2.0)
    
    # Done
    return MSE

In [ ]:
# Run fit with `minimize`

# Initial guess [ki, ko, Nl(0), Cl(0)]
params_0 = [0.1, 0.01, 0.1, 0.9]

# Boundary conditions
bounds = [(10e-6, None), (10e-6, None), (0.0, None), (0.0, None)]

# Run minimization
res = minimize(ratiofit_loss_on, params_0, args=(on_times, on_ratios), bounds=bounds)
params_on = res.x
print('ki={:.4}, ko={:.4}, Nl(0)={:.4}, Cl(0)={:.4}'.format(*params_on))

In [ ]:
# Inspect the result

plt.scatter(on_times, on_ratios, color='teal', alpha=0.4)

pred_t = np.linspace(0, 10, 100)
pred_Nl = Nl_fit_on_np(*params_on, pred_t)
pred_Cl = Cl_fit_on_np(*params_on, pred_t)
pred_R = pred_Nl / (pred_Nl + pred_Cl) * 2

plt.plot(pred_t, pred_R, 
         color='darkblue', alpha=0.7, lw=2)

plt.xlabel('time [min]')
plt.ylabel('Nl/Tl ratios')
plt.show()

In [ ]:
# Convert the result into a function for total nuclear concentration; N(N0, t)

# Combine and insert parameters
N_on_final = Nl_fit_on + Nd_fit_on
N_on_final = N_on_final.subs({ki:params_on[0], ko:params_on[1]})

# We can assume that T(t) = Tl(t) + Td(t) = 1.0
N_on_final = N_on_final.subs({Cl(0) : 1.0 - Nl(0)})

# The total doesn't bleach, so Nl(0)=N(0)
N0 = symbols('N0')
N_on_final = N_on_final.subs({Nl(0) : N0}).simplify()
display(N_on_final)

# Lambdify
N_on_final_np = lambdify([N0, t], N_on_final, modules="numpy")
print('\nN_on_final_npsignature:', inspect.signature(N_on_final_np))

# Pickle it & ship it!
dill.settings['recurse'] = True
with open('../optonotch/modeling/fitted_import_func_sympy_bc.pkl', 'wb') as outfile:
    #pickle.dump(N_on_final_np, outfile) # Pickle won't do it...
    dill.dump(N_on_final_np, outfile)

#### Fitting for the `off` condition (export)

In [ ]:
# Prepare the equations for fitting

# Show  example
display(Nl_rdy.rhs)

# Replace what is known
res_Nl = Nl_fit_on_np(*params_on, 10.0)
res_Cl = Cl_fit_on_np(*params_on, 10.0)
res_Nd = Nd_fit_on_np(*params_on, 10.0)
res_Cd = Cd_fit_on_np(*params_on, 10.0)
Nl_fit_off = Nl_rdy.rhs.subs({Nl(0):res_Nl, Cl(0):res_Cl, Nd(0):res_Nd, Cd(0):res_Cd, b:0.0000})
Cl_fit_off = Cl_rdy.rhs.subs({Nl(0):res_Nl, Cl(0):res_Cl, Nd(0):res_Nd, Cd(0):res_Cd, b:0.0000})
Nd_fit_off = Nd_rdy.rhs.subs({Nl(0):res_Nl, Cl(0):res_Cl, Nd(0):res_Nd, Cd(0):res_Cd, b:0.0000})
Cd_fit_off = Cd_rdy.rhs.subs({Nl(0):res_Nl, Cl(0):res_Cl, Nd(0):res_Nd, Cd(0):res_Cd, b:0.0000})

# Show example result
display(Nl_fit_off)

# Lambdify
Nl_fit_off_np = lambdify([ki, ko, t], Nl_fit_off, modules="numpy")
Cl_fit_off_np = lambdify([ki, ko, t], Cl_fit_off, modules="numpy")
Nd_fit_off_np = lambdify([ki, ko, t], Nd_fit_off, modules="numpy")
Cd_fit_off_np = lambdify([ki, ko, t], Cd_fit_off, modules="numpy")
print('\nNl_fit_off_np signature:', inspect.signature(Nl_fit_off_np))

In [ ]:
# Loss function

def ratiofit_loss_off(params, t, data):
    
    # Run prediction
    pred_Nl = Nl_fit_off_np(*params, t)
    pred_Cl = Cl_fit_off_np(*params, t)
    pred_R = pred_Nl / (pred_Nl + pred_Cl) * 2
    
    # Compute mean square error
    MSE = np.mean((data - pred_R)**2.0)
    
    # Done
    return MSE

In [ ]:
# Run fit with `minimize`

# Initial guess [ki, ko]
params_0 = [0.01, 0.1]

# Boundary conditions
bounds = [(10e-6, None), (10e-6, None)]

# Run minimization
res = minimize(ratiofit_loss_off, params_0, args=(off_times, off_ratios), bounds=bounds)
params_off = res.x
print('ki={:.4}, ko={:.4}'.format(*params_off))

In [ ]:
# Inspect the result

# Prep
plt.figure(figsize=(7,5))

# Plot the `on` for reference
plt.scatter(on_times, on_ratios, 
            color='teal', alpha=0.4, label='$on$ data')
pred_t = np.linspace(0, 10, 100)
pred_Nl = Nl_fit_on_np(*params_on, pred_t)
pred_Cl = Cl_fit_on_np(*params_on, pred_t)
pred_R = pred_Nl / (pred_Nl + pred_Cl) * 2
plt.plot(pred_t, pred_R, 
         color='darkturquoise', alpha=0.8, lw=3, label='$on$ fit')

# Plot the `off`
plt.scatter(off_times+10.0, off_ratios, 
            color='darkorange', alpha=0.4, label='$off$ data')
pred_t = np.linspace(0, 10, 100)
pred_Nl = Nl_fit_off_np(*params_off, pred_t)
pred_Cl = Cl_fit_off_np(*params_off, pred_t)
pred_R = pred_Nl / (pred_Nl + pred_Cl) * 2
plt.plot(pred_t+10.0, pred_R, 
         color='orangered', alpha=0.8, lw=3, label='$off$ fit')

# Legend (properly ordered)
handles, labels = plt.gca().get_legend_handles_labels()
order = [2,0,3,1]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order],
           frameon=True, fontsize=12)

# Finalize
plt.xlabel('time [min]', fontsize = 14)
plt.ylabel('NICD nuclear ratio', fontsize=14)
plt.gca().tick_params(axis='both', which='major', labelsize=13)

plt.savefig(r'../Figures/EV1_A.pdf')

plt.show()

In [ ]:
# Convert the result into a function for total nuclear concentration; N(N0, t)

# Recreate the fit functions without fixed initial conditions
Nl_fit = Nl_rdy.rhs.subs({b:0.0000})
Nd_fit = Nd_rdy.rhs.subs({b:0.0000})

# Combine and insert parameters
N_off_final = Nl_fit + Nd_fit
N_off_final = N_off_final.subs({ki:params_off[0], ko:params_off[1]})

# We can assume that T(t) = Tl(t) + Td(t) = 1.0
N_off_final = N_off_final.subs({Cl(0) : 1.0 - Nl(0)})

# The total doesn't bleach, so Nl(0)=N(0) and Nd(0)=Cd(0)=0
N0 = symbols('N0')
N_off_final = N_off_final.subs({Nl(0):N0, Nd(0):0, Cd(0):0}).simplify()
display(N_off_final)

# Lambdify
N_off_final_np = lambdify([N0, t], N_off_final, modules="numpy")
print('\nN_off_final signature:', inspect.signature(N_off_final_np))

# Pickle it & ship it!
dill.settings['recurse'] = True
with open('../optonotch/modeling/fitted_export_func_sympy_bc.pkl', 'wb') as outfile:
    dill.dump(N_off_final_np, outfile)

#### Double-check the model's outputs

In [ ]:
# Check behavior of concentrations, not ratios!

# Prep
plt.figure(figsize=(12,7))

# Plot the `on`
pred_t = np.linspace(0, 10, 100)
pred_Nl = Nl_fit_on_np(*params_on, pred_t)
pred_Nd = Nd_fit_on_np(*params_on, pred_t)
pred_Cl = Cl_fit_on_np(*params_on, pred_t)
pred_Cd = Cd_fit_on_np(*params_on, pred_t)
pred_N = pred_Nl + pred_Nd
pred_C = pred_Cl + pred_Cd
plt.plot(pred_t, pred_Nl, '--', color='teal', alpha=0.4, lw=2, label='nuc-L')
plt.plot(pred_t, pred_Nd, ':', color='teal', alpha=0.4, lw=2, label='nuc-D')
plt.plot(pred_t, pred_N, color='teal', alpha=0.9, lw=2, label='nuc')
plt.plot(pred_t, pred_Cl, '--', color='darkorange', alpha=0.4, lw=2, label='cyt-L')
plt.plot(pred_t, pred_Cd, ':', color='darkorange', alpha=0.4, lw=2, label='cyt-D')
plt.plot(pred_t, pred_C, color='darkorange', alpha=0.9, lw=2, label='cyt')
plt.plot(pred_t, pred_C + pred_N, '--', color='purple', alpha=0.7, lw=2, label='all')

# Plot the `off`
pred_t = np.linspace(0, 10, 100)
pred_Nl = Nl_fit_off_np(params_off[0], params_off[1], pred_t)
pred_Nd = Nd_fit_off_np(params_off[0], params_off[1], pred_t)
pred_Cl = Cl_fit_off_np(params_off[0], params_off[1], pred_t)
pred_Cd = Cd_fit_off_np(params_off[0], params_off[1], pred_t)
pred_N = pred_Nl + pred_Nd
pred_C = pred_Cl + pred_Cd
plt.plot(pred_t+10.0, pred_Nl, '--', color='teal', alpha=0.4, lw=2)
plt.plot(pred_t+10.0, pred_Nd, ':', color='teal', alpha=0.4, lw=2)
plt.plot(pred_t+10.0, pred_N, color='teal', alpha=0.9, lw=2)
plt.plot(pred_t+10.0, pred_Cl, '--', color='darkorange', alpha=0.4, lw=2)
plt.plot(pred_t+10.0, pred_Cd, ':', color='darkorange', alpha=0.4, lw=2)
plt.plot(pred_t+10.0, pred_C, color='darkorange', alpha=0.9, lw=2)
plt.plot(pred_t+10.0, pred_C + pred_N, '--', color='purple', alpha=0.7, lw=2)

# Add the light
plt.plot([0, 10], [1.0, 1.0], '-', color='royalblue', lw=4, label='light on')

# Finalize
plt.ylim(-0.1, 1.1)
plt.legend(ncol=4)
plt.xlabel('time [min]', fontsize = 12)
plt.ylabel('nuclear concentration', fontsize=12)
plt.gca().tick_params(axis='both', which='major', labelsize=12)
plt.show()

# Noice.